This is a Python script that parses the output from replication study for cross-version (make sure the output folders are in the same directory as this file), extracting the MCC values.

In [6]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict


In [7]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [25]:
disagreementCols = ['SC', 'Bagging*MCC', 'Boosting*MCC', 
 'Stacking_by_DIV*MCC',  'Stacking_by_MCC*MCC', 'Stacking_by_PRECISION*MCC', 'Stacking_by_WAD*MCC', 'Bagging*PREC', 'Boosting*PREC', 'Stacking_by_DIV*PREC', 'Stacking_by_MCC*PREC', 'Stacking_by_PRECISION*PREC',  'Stacking_by_WAD*PREC', 'NB', 'J48', 'KNN', 'SMO']

otherDivCols = ['Stacking_by_DIV*MCC', 'Stacking_by_WAD*MCC', 'Stacking_by_DIV*PREC', 'Stacking_by_WAD*PREC']

In [26]:
def generate_csv_mcc_values(div, experiment, projects):
    mccs = defaultdict(list)      
    
    print(f'Diversity: {div}\tVariant: {experiment}')
    for p in projects:
        print(f'Project: {p}')
        for dirname, _, filenames in os.walk(f'./{div}/{experiment}'):
            for filename in filenames:
                path = os.path.join(dirname, filename)
                split = path.split('/')
                clf = split[len(split)-2]
                project = split[len(split)-1]

                if project != p: continue

                # print(f'Classifier: {clf}')

                if clf == 'Base':
                    # parse base in a different way
                    parseNext = False
                    i = 0
                    with open(path) as f:
                        for line in f:
                            line = line.rstrip()
                            if len(line) == 0: continue
                            if line == '============ OVERALL MCC STATS ==============':
                                parseNext = True
                            elif parseNext and isfloat(line):
                                # parse mcc
                                if i == 0:
                                    mccs['SC'].append(float(line))
                                if i == 1:
                                    mccs['NB'].append(float(line))
                                if i == 2:
                                    mccs['J48'].append(float(line))
                                if i == 3:
                                    mccs['KNN'].append(float(line))  
                                if i == 4:
                                    mccs['SMO'].append(float(line))   
                                parseNext = False
                                i += 1
                else:
                    parseNext = False
                    i = 0
                    prevMeta = ''
                    with open(path) as f:
                        for line in f:
                            line = line.rstrip()
                            if len(line) == 0: continue
                            if "with meta" in line:
                                # get the meta classifier
                                meta = line.split(':')[1].strip()
                            elif line == '============ OVERALL MCC STATS ==============':
                                parseNext = True
                            elif parseNext and isfloat(line):
                                # parse mcc
                                if prevMeta == meta:
                                    # same meta clf

                                    # for bagging and boosting classifers output, output file always contains results for 
                                    # best classifier by MCC followed by best classifier by PRECISION as the logic for skipping
                                    # was broken when the scripts ran. 
                                    mccs[clf+("*MCC" if i==0 else "*PREC")].append('x')
                                else:
                                    mccs[clf+("*MCC" if i==0 else "*PREC")].append(float(line))
                                parseNext = False
                                prevMeta = meta
                                i += 1
                    if i == 1:
                        # this is only used for stacking classifiers
                        # as the logic for skipping work if best classifier by MCC == best classifier by PRECISION was working
                        # correctly, unlike bagging and boostingdef isfloat(value):
                        mccs[clf+"*PREC"].append('x')

    # create a pandas dataframe
    df = pd.DataFrame(mccs, index=projects)
    
    # rearrange columns to match order in the sheet uploaded on GDrive
    if div == 'DISAGREEMENT':
        df = df[disagreementCols]
    else:
        df = df[otherDivCols]

    df.to_csv(f'results/{div}/{experiment}.csv', encoding='utf-8', index=True, header=True)

In [27]:
diversities = ['DISAGREEMENT', 'CORRELATION', 'DOUBLE_FAULT', 'QSTATISTIC']
experiments = ['firstLast','penultimate-last']
projects = ['activemq', 'camel', 'derby', 'groovy', 'hbase', 'hive', 'jruby', 'lucene', 'wicket']

for div in diversities:
    for experiment in experiments:
        # temporarily remove camel-penultimate from QSTATISTIC as it is missing some data
        if div == 'QSTATISTIC':
            projects = list(filter(lambda p: p != 'camel', projects))
        generate_csv_mcc_values(div, experiment, projects)

Diversity: DISAGREEMENT	Variant: firstLast
Project: activemq
Project: camel
Project: derby
Project: groovy
Project: hbase
Project: hive
Project: jruby
Project: lucene
Project: wicket
Diversity: DISAGREEMENT	Variant: penultimate-last
Project: activemq
Project: camel
Project: derby
Project: groovy
Project: hbase
Project: hive
Project: jruby
Project: lucene
Project: wicket
Diversity: CORRELATION	Variant: firstLast
Project: activemq
Project: camel
Project: derby
Project: groovy
Project: hbase
Project: hive
Project: jruby
Project: lucene
Project: wicket
Diversity: CORRELATION	Variant: penultimate-last
Project: activemq
Project: camel
Project: derby
Project: groovy
Project: hbase
Project: hive
Project: jruby
Project: lucene
Project: wicket
Diversity: DOUBLE_FAULT	Variant: firstLast
Project: activemq
Project: camel
Project: derby
Project: groovy
Project: hbase
Project: hive
Project: jruby
Project: lucene
Project: wicket
Diversity: DOUBLE_FAULT	Variant: penultimate-last
Project: activemq
Proje